## AABC cumulative recruitment stats and visuals


In [ ]:
# run all and then, from the command line:
#jupyter nbconvert --to html --no-input AABC_HCA_recruitmentstats.ipynb --html report.html


In [ ]:
#load some libraries
import pandas as pd
import seaborn as sns
from ccf.box import LifespanBox
import yaml
from functions import *
from config import *
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

In [ ]:
print(date.today().strftime("%m/%d/%Y"))

In [ ]:
#load HCA inventory 
config = LoadSettings()
secret=pd.read_csv(config['config_files']['secrets'])
box = LifespanBox(cache="./tmp")
pathp=box.downloadFile(config['hcainventory'])
ids=pd.read_csv(pathp)

In [ ]:
#load AABC report
aabcarms = redjson(tok=secret.loc[secret.source=='aabcarms','api_key'].reset_index().drop(columns='index').api_key[0])
hcpa = redjson(tok=secret.loc[secret.source=='hcpa','api_key'].reset_index().drop(columns='index').api_key[0])
#just a report
aabcreport = redreport(tok=secret.loc[secret.source=='aabcarms','api_key'].reset_index().drop(columns='index').api_key[0],reportid='51031')

#download the inventory report from AABC for comparison
aabcinvent=getframe(struct=aabcreport,api_url=config['Redcap']['api_url10'])


In [ ]:
aabcinvent['todaydate']=date.today()
aabcinvent['dayspassed']=(pd.to_datetime(aabcinvent.todaydate) - pd.to_datetime(aabcinvent.v0_date)).dt.days


#get list of subjects who passed recruitment and registered for a visit
#for i in list(aabcinvent.columns):
#    print(i)
    
subjects=aabcinvent[['study_id','subject_id','redcap_event_name','passedscreen','register_subject_complete','register_visit_complete']]
#print(subjects.shape)
subjects=subjects.loc[subjects.register_visit_complete =='2'][['study_id']]
subs=list(subjects.study_id)


In [ ]:

forplot=aabcinvent.loc[(aabcinvent.study_id.isin(subs)) & (aabcinvent.redcap_event_name.str.contains('register'))][['v0_date','dayspassed','age','sex','racial','ethnic','site','counterbalance_1st']]
forplot=forplot.sort_values('dayspassed')

#forplot.head()

In [ ]:
#PREPARE DATA FOR PLOTS
#forplot['sexsum']=
S=pd.get_dummies(forplot.sex, prefix='sex')

#forplot['sexsum']=pd.to_numeric(forplot.sex, errors='coerce').cumsum()
forplot['malesum']=pd.to_numeric(S.sex_1, errors='coerce').cumsum()
forplot['femalesum']=pd.to_numeric(S.sex_2, errors='coerce').cumsum()
forplot['Sex']=forplot.sex.replace({'1':'Male','2':'Female'})

S0=pd.get_dummies(forplot.counterbalance_1st, prefix='CB')
forplot['CB3sum']=pd.to_numeric(S0.CB_3, errors='coerce').cumsum()
forplot['CB4sum']=pd.to_numeric(S0.CB_4, errors='coerce').cumsum()
forplot['Counterbalance']=forplot.counterbalance_1st.replace({'3':'CB3','4':'CB4'})

#forplot.head()

In [ ]:
#1, Native American/Alaskan Native | 2, Asian | 3, Black or African American | 4, Native Hawaiian or Other Pacific Is | 5, White | 6, More than one race | 99, Unknown or Not reported
S2=pd.get_dummies(forplot.racial, prefix='race')
#print(S2.head())
forplot['whitesum']=pd.to_numeric(S2.race_5, errors='coerce').cumsum()
#forplot['natpacsum']=pd.to_numeric(S2.race_4, errors='coerce').cumsum()
forplot['blacksum']=pd.to_numeric(S2.race_3, errors='coerce').cumsum()
forplot['asiansum']=pd.to_numeric(S2.race_2, errors='coerce').cumsum()
forplot['natamersum']=pd.to_numeric(S2.race_1, errors='coerce').cumsum()
forplot['moret1sum']=pd.to_numeric(S2.race_6, errors='coerce').cumsum()
#forplot['nasum']=pd.to_numeric(S2.race_7, errors='coerce').cumsum()
forplot['Race']=forplot.racial.replace({'1':'Nat Amer/Alaskan','2':'Asian','3':'Black','4':'Nat Hawaiian/PI','5':'White','6':'More than one','7':'Unknown'})

#thnicity
S3=pd.get_dummies(forplot.ethnic, prefix='ethnicity')
forplot['hispanicsum']=pd.to_numeric(S3.ethnicity_1, errors='coerce').cumsum()
forplot['nonhispanicsum']=pd.to_numeric(S3.ethnicity_2, errors='coerce').cumsum()
forplot['unkhispsum']=pd.to_numeric(S3.ethnicity_3, errors='coerce').cumsum()
forplot['Ethnicity']=forplot.ethnic.replace({'1':'Hispanic','2':'Non-Hispanic','3':'Unknown'})

#sites
S4=pd.get_dummies(forplot.site, prefix='site')
forplot['wusum']=pd.to_numeric(S4.site_4, errors='coerce').cumsum()
forplot['umnsum']=pd.to_numeric(S4.site_3, errors='coerce').cumsum()
forplot['mghsum']=pd.to_numeric(S4.site_1, errors='coerce').cumsum()
forplot['uclasum']=pd.to_numeric(S4.site_2, errors='coerce').cumsum()
forplot['Site']=forplot.site.replace({'1':'MGH','2':'UCLA','3':'UMN','4':'WashU'})

In [ ]:
                                       
##ages
bins= [30,40,50,60,70,80,90,120]
forplot['ages']=pd.to_numeric(forplot.age)
forplot['AgeGroup'] = pd.cut(forplot['ages'], bins=bins,right=False)# labels=labels,
S5=pd.get_dummies(forplot.AgeGroup, prefix='age')
forplot['age30sum']=pd.to_numeric(S5['age_[30, 40)'], errors='coerce').cumsum()
forplot['age40sum']=pd.to_numeric(S5['age_[40, 50)'], errors='coerce').cumsum()
forplot['age50sum']=pd.to_numeric(S5['age_[50, 60)'], errors='coerce').cumsum()
forplot['age60sum']=pd.to_numeric(S5['age_[60, 70)'], errors='coerce').cumsum()
forplot['age70sum']=pd.to_numeric(S5['age_[70, 80)'], errors='coerce').cumsum()
forplot['age80sum']=pd.to_numeric(S5['age_[80, 90)'], errors='coerce').cumsum()
forplot['age90sum']=pd.to_numeric(S5['age_[90, 120)'],errors='coerce').cumsum()
#forplot.columns

## AABC Univariate Counts

In [ ]:
### create plot of AABC recruitment stats by SEX
# Create data
x=list(forplot.dayspassed) #range(1,6)
y1=list(forplot.malesum) #[1,4,6,8,9]
y2=list(forplot.femalesum)#[2,2,7,10,12]

# Basic stacked area chart.
plt.stackplot(x,y1, y2, labels=['Male:'+str(max(y1)),'Female:'+str(max(y2))])
plt.legend(loc='upper left')
plt.xlabel('Days Passed Since 1st Recruit')
plt.ylabel('Number of Subjects')



In [ ]:
### create plot of AABC recruitment stats by SEX
# Create data
x=list(forplot.dayspassed) #range(1,6)
y1=list(forplot.CB3sum) #[1,4,6,8,9]
y2=list(forplot.CB4sum)#[2,2,7,10,12]

# Basic stacked area chart.
plt.stackplot(x,y1, y2, labels=['CB3:'+str(max(y1)),'CB4:'+str(max(y2))])
plt.legend(loc='upper left')
plt.xlabel('Days Passed Since 1st Recruit')
plt.ylabel('Number of Subjects')

In [ ]:
#BY RACE
x=list(forplot.dayspassed) #range(1,6)
y1=list(forplot.whitesum) #[1,4,6,8,9]
y2=list(forplot.blacksum)#[2,2,7,10,12]
y3=list(forplot.asiansum)
y4=list(forplot.moret1sum)
#y5=list(forplot.natpacsum)
y6=list(forplot['natamersum'])
#y7=list(forplot['nasum'])



# Basic stacked area chart.
plt.stackplot(x,y1,y2,y3,y4,y6,labels=['White:'+str(max(y1)),'Black:'+str(max(y2)),'Asian:'+str(max(y3)),'More than one Race:'+str(max(y4)),'Nat Hawiaan/PI:'+str(max(y5)),'Nat American/Alaskan:'+str(max(y6))])
plt.legend(loc='upper left')
plt.xlabel('Days Passed Since 1st Recruit')
plt.ylabel('Number of Subjects')

In [ ]:
#BY ETHNICITY
x=list(forplot.dayspassed) #range(1,6)
y1=list(forplot.nonhispanicsum) #[1,4,6,8,9]
y2=list(forplot.hispanicsum)#[2,2,7,10,12]
y3=list(forplot.unkhispsum)
# Basic stacked area chart.
plt.stackplot(x,y1, y2,y3, labels=['Non-Hispanic:'+str(max(y1)),'Hispanic:'+str(max(y2)),'Unknown or Not Reported:'+str(max(y3))])
plt.legend(loc='upper left')
plt.xlabel('Days Passed Since 1st Recruit')
plt.ylabel('Number of Subjects')

In [ ]:
#By Site
x=list(forplot.dayspassed) #range(1,6)
y1=list(forplot.wusum) #[1,4,6,8,9]
y2=list(forplot.umnsum)#[2,2,7,10,12]
y3=list(forplot.mghsum)
y4=list(forplot.uclasum)
# Basic stacked area chart.
plt.stackplot(x,y1, y2,y3, y4,labels=['WU:'+str(max(y1)),'UMN:'+str(max(y2)),'MGH:'+str(max(y3)),'UCLA:'+str(max(y4))])
plt.legend(loc='upper left')
plt.xlabel('Days Passed Since 1st Recruit')
plt.ylabel('Number of Subjects')


In [ ]:
#By Age Bin
x=list(forplot.dayspassed) #range(1,6)
y2=list(forplot.age30sum)#[2,2,7,10,12]
y3=list(forplot.age40sum)
y4=list(forplot.age50sum)
y5=list(forplot.age60sum)
y6=list(forplot.age70sum)
y7=list(forplot.age80sum)
y8=list(forplot.age90sum)

# Basic stacked area chart.
plt.stackplot(x,y2,y3,y4,y5,y6,y7,y8, labels=['Age [30-40):'+str(max(y2)),'Age [40-50):'+str(max(y3)),'Age [50-60):'+str(max(y4)),'Age [60-70):'+str(max(y5)),'Age [70-80):'+str(max(y6)),'Age [80-90):'+str(max(y7)),'Age [90+):'+str(max(y8))])
plt.legend(loc='upper left')

plt.xlabel('Days Passed Since 1st Recruit')
plt.ylabel('Number of Subjects')

## AABC Crosstabulations


In [ ]:
#Crosstabs x Site

pd.crosstab(forplot.Race,forplot.Site).plot.bar(rot=45)
#pd.crosstab(forplot.Race,forplot.Site).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.Race,forplot.Site)

In [ ]:
pd.crosstab(forplot.Ethnicity,forplot.Site).plot.bar(rot=45)
#pd.crosstab(forplot.Ethnicity,forplot.Site).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.Ethnicity,forplot.Site)

In [ ]:
pd.crosstab(forplot.AgeGroup,forplot.Site).plot.bar(rot=45)
#pd.crosstab(forplot.AgeGroup,forplot.Site).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.AgeGroup,forplot.Site)

In [ ]:
pd.crosstab(forplot.Sex,forplot.Site).plot.bar(rot=45)
#pd.crosstab(forplot.Sex,forplot.Site).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.Sex,forplot.Site)

In [ ]:
pd.crosstab(forplot.AgeGroup,forplot.Race).plot.bar(rot=45)
#pd.crosstab(forplot.AgeGroup,forplot.Race).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.AgeGroup,forplot.Race)

In [ ]:
pd.crosstab(forplot.AgeGroup,forplot.Ethnicity).plot.bar(rot=45)
#pd.crosstab(forplot.AgeGroup,forplot.Ethnicity).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.AgeGroup,forplot.Ethnicity)


In [ ]:
pd.crosstab(forplot.AgeGroup,forplot.Sex).plot.bar(rot=45)
#pd.crosstab(forplot.AgeGroup,forplot.Sex).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.AgeGroup,forplot.Sex)

In [ ]:
#Crosstabs
pd.crosstab(forplot.Race,forplot.Sex).plot.bar(rot=45)
#pd.crosstab(forplot.Race,forplot.Sex).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.Race,forplot.Sex)

In [ ]:
#Crosstabs
pd.crosstab(forplot.Ethnicity,forplot.Sex).plot.bar(rot=45)
#pd.crosstab(forplot.Ethnicity,forplot.Sex).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.Ethnicity,forplot.Sex)

In [ ]:
#Crosstabs
pd.crosstab(forplot.Counterbalance,forplot.Sex).plot.bar(rot=0)
#pd.crosstab(forplot.Counterbalance,forplot.Sex).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.Counterbalance,forplot.Sex)

In [ ]:
#Crosstabs
pd.crosstab(forplot.Race,forplot.Counterbalance).plot.bar(rot=45)
#pd.crosstab(forplot.Race,forplot.Counterbalance).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.Race,forplot.Counterbalance)

In [ ]:
#Crosstabs
pd.crosstab(forplot.AgeGroup,forplot.Counterbalance).plot.bar(rot=45)
#pd.crosstab(forplot.AgeGroup,forplot.Counterbalance).to_csv('Recruitment_Stats',mode='a')
pd.crosstab(forplot.AgeGroup,forplot.Counterbalance)

In [ ]:
## UNIVARIATE HCA demographic stats

In [ ]:
#request or load REDCAP API Key

In [ ]:
##subset to v1 for recruitment stats
#forstats=ids[['subject','redcap_event','event_age','site','M/F','race','ethnic_group','pedid']].loc[ids.redcap_event.isin(['V1','V2'])].sort_values('redcap_event').drop_duplicates(subset='subject',keep='first')


In [ ]:
#bin ages
#bins= [20,30,40,50,60,70,80,90,100,110]
#forstats['AgeGroup'] = pd.cut(forstats['event_age'], bins=bins,right=False)# labels=labels,

In [ ]:
#a=pd.DataFrame(forstats.groupby(['site']).count().pedid).rename(columns={'pedid':'count'})
#a.to_csv('HCA_Site.csv')
#b=pd.DataFrame(forstats.groupby(['AgeGroup']).count().pedid).rename(columns={'pedid':'count'})
#b.to_csv('HCA_Age.csv')
#c=pd.DataFrame(forstats.groupby(['M/F']).count().pedid).rename(columns={'pedid':'count'})
#c.to_csv('HCA_Sex.csv')
#d=pd.DataFrame(forstats.groupby(['race']).count().pedid).rename(columns={'pedid':'count'})
#d.to_csv('HCA_Race.csv')
#e=pd.DataFrame(forstats.groupby(['ethnic_group']).count().pedid).rename(columns={'pedid':'count'})
#e.to_csv('HCA_Ethnicity.csv')


In [ ]:
## HCA demographics by Age

In [ ]:
#print('************  BY AGE  ********************')
#print(forstats.groupby(['AgeGroup','site']).count().pedid)
#pd.DataFrame(pd.crosstab(forstats['AgeGroup'], [forstats.site])).to_csv('HCA_Age_x_Site.csv',index=True)
#print('*******************************************************')
#print(forstats.groupby(['AgeGroup','M/F']).count().pedid)
#pd.DataFrame(pd.crosstab(forstats['AgeGroup'], forstats['M/F'])).to_csv('HCA_Age_x_Sex.csv',index=True)
#print('*******************************************************')
#print(forstats.groupby(['AgeGroup','race']).count().pedid)
#pd.DataFrame(pd.crosstab(forstats['AgeGroup'], [forstats.race])).to_csv('HCA_Age_x_Race.csv',index=True)
#print('*******************************************************')
#print(forstats.groupby(['AgeGroup','ethnic_group']).count().pedid)
#pd.DataFrame(pd.crosstab(forstats['AgeGroup'], [forstats.ethnic_group])).to_csv('HCA_Age_x_Ethnicity.csv',index=True)
#print('*******************************************************')

In [ ]:
# # HCA demographics by Site

In [ ]:
#print('************* BY SITE *********************')
#print(forstats.groupby(['site','M/F']).count().pedid)
#pd.DataFrame(pd.crosstab(forstats['site'], [forstats['M/F']])).to_csv('HCA_Site_x_Sex.csv',index=True)
#print('*******************************************************')
#print(forstats.groupby(['site','race']).count().pedid)
#pd.DataFrame(pd.crosstab(forstats['site'], [forstats.race])).to_csv('HCA_Site_x_Race.csv',index=True)
#print('*******************************************************')
#print(forstats.groupby(['site','ethnic_group']).count().pedid)
#pd.DataFrame(pd.crosstab(forstats['site'], [forstats.ethnic_group])).to_csv('HCA_Site_x_Ethnicity.csv',index=True)
#print('*******************************************************')
#
#print('************ BY SEX **********************')
#pd.DataFrame(pd.crosstab(forstats['M/F'], [forstats['race']])).to_csv('HCA_Sex_x_Race.csv',index=True)
#print('*******************************************************')
#pd.DataFrame(pd.crosstab(forstats['M/F'], [forstats['ethnic_group']])).to_csv('HCA_Sex_x_Ethnicity.csv',index=True)
#print('*******************************************************')

#print('************ BY RACE **********************')
#pd.DataFrame(pd.crosstab(forstats['race'], [forstats['ethnic_group']])).to_csv('HCA_Race_x_Ethnicity.csv',index=True)
#print('*******************************************************')